# DNBLab Jupyter Notebook Tutorial

## SRU-Abfragen erklärt - Tutorial für Einsteiger, die es wissen wollen

Dieses Tutorial erklärt die Möglichkeiten, mit Hilfe von Jupyter Notebooks und Python die SRU-Schnittstelle der DNB abzufragen und mit den erhaltenen Antworten zu arbeiten. Das Tutorial erklärt dazu den Aufbau der Abfragen anhand von Beispielen und stützt sich auf die Dokumentation der SRU-Schnittstelle unter https://www.dnb.de/sru.

Das Tutorial ist dazu in folgendermaßen aufgebaut: 

* [1. Einrichten der Arbeitsumgebung](#Teil1) 
* [2. Abfragen verschiedener Datensätze der DNB](#Teil2)  
* [3. Aufbau einer gezielten Suche](#Teil3) 
* [4. Verarbeiten der Ergebnisse](#Teil4)


### 1. Einrichten der Arbeitsumgebung  <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, werden zunächst die benötigten Python-Biblitoheken importiert: "requests" für die Abfragen an die SRU-Schnittstelle, ElementTree (als ET) und BeautifulSoup, um die XML-Antworten der Schnittstelle besser verarbeiten zu können und unicodedata zur Codierung:

In [19]:
import requests
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

#URL der SRU-Schnittstelle der DNB: 
base_url = "https://services.dnb.de/sru"

#Anfrage - wir speichern das Ergebnis in die Variable "basic_request":
basic_request = requests.get(base_url)

Die Antwort der Schnittstelle kann auf verschiedenen Wegen betrachtet werden: So kann bspw. der übergebene Inhalt als Text ausgegeben werden. Dies ist vor allem dann sinnvoll, wenn unklar ist, welches Format die Schnittstelle per default ausliefert. Alternativ kann der Inhalt natürlich auch direkt in das passende Format konvertiert werden. 

In diesem Fall gibt die SRU-Schnittstelle zunächst HTML zurück:

In [20]:
#Ausgabe der ursprünglichen Abfrage als Text: 
print(basic_request.text)

<html>
<body>
<h1>SRU is running.</h1>
Version: 2.31, Build: 2022-03-01T12:10:32 UTC
</body>
</html>



### 2. Abfragen verschiedener Datensätze der DNB  <a class="anchor" id="Teil2"></a>

Die DNB bietet Ihre Daten auf drei verschiedene "Kataloge" gesplittet an, von denen immer einer für eine Abfrage ausgewählt werden muss. Dies geschieht über eine Erweiterung der Base-URL. Zur Verfügung stehen folgende Kataloge: 

* Katalog der Deutschen Nationalbibliothek (DNB) - hierin befinden sich die Titeldaten
* Katalog des Deutschen Musikarchivs (DMA) - Datensätze des Deutschen Musikarchivs
* Katalog der Gemeinsamen Normdatei (GND) - hierin befinden sich die Normdaten

Die erweiterungen für die URL sind folgende: 

* DNB: https://services.dnb.de/sru/dnb
* DMA: https://services.dnb.de/sru/dnb.dma
* GND: https://services.dnb.de/sru/authorities

Werden die jeweiligen Bereiche ohne weitere Spezifikationen abgefragt, senden sie eine "Explain-Response" in XML zurück. 

Mit Hilfe der Bibliothek BeautifulSoup kann die Antwort direkt in XML umgewandelt werden. 

Hinweis: Um die Antwort nicht zu lang werden zu lassen, werden hier nur die ersten 500 Zeichen der Antwort ausgegeben - die eigentliche Antwort ist länger und kann durch einfaches Löschen der Einschränkung "[0:500]" in der "print"-Zeile komplett angezeigt werden. Natürlich können auch andere Bereiche zur Anzeige gewählt werden. 

In [21]:
dnb = requests.get("https://services.dnb.de/sru/dnb")

response = soup(dnb.content, features="xml")
print(response.prettify()[0:500])


<?xml version="1.0" encoding="utf-8"?>
<explainResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <record>
  <recordSchema>
   http://explain.z3950.org/dtd/2.0/
  </recordSchema>
  <recordPacking>
   xml
  </recordPacking>
  <recordData>
   <ns:explain id="Deutsche Nationalbibliothek" xmlns:ns="http://explain.z3950.org/dtd/2.0/">
    <ns:serverInfo protocol="sru" version="1.1">
     <ns:host>
      services.dnb.de
     </ns:host>
     <ns:port>
      443
     </ns:port>


Für eine Suchanfrage an die Daten der DNB wird nun zunächst über die Wahl der URL der Katalog definiert. Mit Hilfe der Variable *parameter* werdem dann alle weiteren Parameter, die die SRU-Schnittstelle benötigt, übergeben. 

Besonders relevant sind im Folgenden dabei die beiden Punkte 'query' : 'Klimawandel', sowie 'recordSchema' : 'MARC21-xml'. Statt "Klimawandel" kann hier jeder beliebige Suchbegriff eingetragen werden - auch Suchbegriffe, die aus mehreren Wörtern bestehen, können hier mittels boolscher Operatoren übergeben werden. Wie genau solche Anfragen formuliert werden müssen, kann unter https://www.dnb.de/sru nachgelesen werden. Statt "MARC21-xml" stehen außerdem noch oai_dc oder RDFxml zur Verfügung (siehe etwas weiter unten).   


In [5]:
dnb_url = "https://services.dnb.de/sru/dnb"

#Parameter, die wir mit einer Anfrage übergeben wollen: 
parameter = {'version' : '1.1' , 
             'operation' : 'searchRetrieve' , 
             'query' : 'Klimawandel', 
             'recordSchema' : 'MARC21-xml'} 

r1 = requests.get(dnb_url, params = parameter)

#print(r.url)

response = soup(r1.content, features="xml")
print(response.prettify()[0:1000])


<?xml version="1.0" encoding="utf-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <numberOfRecords>
  9939
 </numberOfRecords>
 <records>
  <record>
   <recordSchema>
    MARC21-xml
   </recordSchema>
   <recordPacking>
    xml
   </recordPacking>
   <recordData>
    <record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
     <leader>
      00000nam a22000008c 4500
     </leader>
     <controlfield tag="001">
      1147699615
     </controlfield>
     <controlfield tag="003">
      DE-101
     </controlfield>
     <controlfield tag="005">
      20221206110102.0
     </controlfield>
     <controlfield tag="007">
      tu
     </controlfield>
     <controlfield tag="008">
      171204s2027    gw ||||| |||| 00||||ger
     </controlfield>
     <datafield ind1=" " ind2=" " tag="015">
      <subfield code="a">
       17,N50
      </subfield>
      <subfield code="2">
       dnb
      </subfield>
     </datafield>
     <datafield 

Zu beachten ist, dass die Suche nach einem Stichwort über den 'query'-Befehl eine allgemeine Suche über alle Titeldaten darstellt. Die Suche ist allerdings nicht auf Titel oder ähnliches beschränkt, sondern durchsucht die Datensätze im Gesamten. Auch nach beispielsweise Autor\*innennamen kann auf diese Art gesucht werden, jedoch muss bedacht werden, dass auch Titel, die den gesuchten Namen enthalten, in diesem Fall als Treffer ausgegeben werden. 

### 3. Aufbau einer gezielten Suche  <a class="anchor" id="Teil3"></a>

Um die Suche direkt auf bestimmte Angaben wie bspw. Titel oder Autor\*in einzugrenzen, können unter anderem folgende Befehle genutzt werden:

 - tit= Suche im Titeleintrag
 - atr= Suche nach Verfasser\*in (Person oder Organisation)
 - per= Suche nach Personen (in allen relevanten Feldern)
 - sw = Suche nach Schlagworten
 - jhr = Suche nach Erscheinungszeitraum 
 - ...

Eine detaillierte Übersicht über die verschiedenen Abfragemöglichkeiten gibt es hier: https://www.dnb.de/expertensuche. Dabei können die unterschiedlichen Parameter auch beliebig in der Suchanfrage kombiniert werden - zu beachten ist hier lediglich, dass diese immer Teil der "Query" sind. 

Für die Ausgabe der Ergebnisse kann außerdem zwischen drei Formaten gewählt werden, indem der entsprechende Code hinter 'recordSchema' geändert wird:  

 - MARC21-xml (XML-Variante von MARC 21)
 - oai_dc (DNB Casual - Auswahl von Dublin-Core-Elementen - nur für Titeldaten!)
 - RDFxml (RDF - Linked Data Service)

Eine Suchanfrage nach Titeln, die das Suchwort "Klimawandel" enthalten und im Jahr 2005 erschienen sind und die im Format DNB Casual ausgegeben werden soll, sieht dann folgendermaßen aus: 

In [6]:
#Parameter, die wir mit einer Anfrage übergeben wollen: 
parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : 'tit=Klimawandel and jhr=2005',
             'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

r = requests.get(dnb_url, params = parameter)

#Parsen der Antwort "r" als XML in die neue Variable "response":
response = soup(r.content, features="xml")

#Schöne Ausgabe der ersten 1000 Zeichen: 
print(response.prettify()[0:1000])


<?xml version="1.0" encoding="utf-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <numberOfRecords>
  29
 </numberOfRecords>
 <records>
  <record>
   <recordSchema>
    oai_dc
   </recordSchema>
   <recordPacking>
    xml
   </recordPacking>
   <recordData>
    <dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
     <dc:title>
      Anpassung an den Klimawandel : Gründe, Folgen, Handlungsoptionen / Bundesministerium für Wirtschaftliche Zusammenarbeit und Entwicklung ; Gtz, Deutsche Gesellschaft für Technische Zusammenarbeit (GTZ) GmbH
     </dc:title>
     <dc:creator>
      Deutschland / Bundesministerium für Wirtschaftliche Zusammenarbeit und Entwicklung
     </dc:creator>
     <dc:publisher>
      Eschborn : GTZ
     </dc:publisher>
     <dc:date>
      2005
     </dc:date>
     <dc:iden

In Zeile 8 der Antwort befindet sich folgender XML-Block, der im Text zwischen den XML-Tags die Gesamtzahl der gefundenen Ergebnisse verrät:  
```
<numberOfRecords>
   ZAHL
</numberOfRecords>
```

Wenn man diese Information nicht unbedingt im XML suchen möchte, kann natürlich auch der Code so angepasst werden, dass der entsprechende Abschnitt mithilfe des Zusatzes ".find('numberOfRecords')" gesucht wird. Der Zusatz ".find('suchtext')" sucht dabei nach dem ersten Element, welches den Suchtext in Klammern enthält, was für diesen Fall ausreichend ist, da diese Information nur einmal pro Anfrage von der SRU-Schnittstelle zurückgegeben wird. Wenn dagegen mehrere XML-Tags mit demselben Namen gesucht und ausgegeben werden sollen, nutzt man ".find_all('suchtext')". 

Im Anschluss kann der Inhalt zwischen den beiden "numberOfRecords"-Tags ausgegeben werdem, indem das Attribut ".text" an die Variable *number* angehängt wird. Zum Vergleich kann die gesamte Variable "number" (unten auskommentiert) ausgegeben werden:

In [17]:
number = response.find('numberOfRecords')
print(number.text, 'Ergebnisse')
#print(number)


29 Ergebnisse


Da die einzelzen Treffer bzw. Werke jeweils durch "record"-Tags gekennzeichnet sind, werden diese nun gesucht und in der Variable *records* zwischengespeichert. Zum Vergleich wird im Anschluss noch die Länge der Variable ausgeben, um überprüfen zu können, ob diese mit der Angabe unter "numberOfRecords" übereinstimmt. 

HINWEIS: Dies funkioniert zunächst nur bis zu einer Treffermenge von insgesamt 100! Auch bei größeren Treffermengen wird im folgenden maximal die Länge 100 angezeigt - wie man trotzdem größere Treffermengen sammeln kann, folgt etwas weiter unten in diesem Tutorial. 

In [9]:
records = response.find_all('record')
print(len(records), 'Ergebnisse')

29 Ergebnisse


Die Ergebnisse werden als Liste gespeichert, was bedeutet, dass die Variable *records* entsprechend eine Listenvariable ist. Die Ergebnisse stehen dabei jeweils an einem eigenen Listenplatz - bei 9 Ergebnissen gibt es in der Liste also 9 Einträge. Da der erste Eintrag allerdings an Listenplatz 0 steht, sind die Plätze von 0-8 mit den 9 Einträgen belegt - dies ist wichtig, wenn einzelene Listenplätze adressieren werden sollen. 

Der 3. Eintrag kann daher bspw. angezeigt werden, indem Listenplatz Nummer 2 aufrufen wird: 

In [18]:
print(records[2])

<record><recordSchema>oai_dc</recordSchema><recordPacking>xml</recordPacking><recordData><dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
<dc:title>Die EU im Einsatz gegen den Klimawandel : der EU-Emissionshandel - ein offenes System, das weltweit Innovationen fördert / [Europäische Kommission]</dc:title>
<dc:creator>Europäische Kommission</dc:creator>
<dc:publisher>[Luxemburg] : [Amt für Amtliche Veröff. der Europ. Gemeinschaften]</dc:publisher>
<dc:date>2005</dc:date>
<dc:language>ger</dc:language>
<dc:identifier xmlns:tel="http://krait.kb.nl/coop/tel/handbook/telterms.html" xsi:type="tel:ISBN">92-894-9187-6 geh.</dc:identifier>
<dc:identifier xsi:type="dnb:IDN">992017882</dc:identifier>
<dc:subject>360 Soziale Probleme, Sozialdienste, Versicherungen</dc:subject>
<dc:subject>330 Wirtschaft</dc:subject>
<dc:format>20 S.</dc:format>
</d

Um das Ganze nun etwas bequemer nutzen zu können, können alle diese Schritte in einer Funktion zusammengeführt werden, die dann nur noch die jeweilige Abfrage übergeben bekommt: 

In [11]:
#Funtion
def sru_dnb(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content, features="xml")
    records = response.find_all('record')  
    
    return records #Rückgabe der Ergebnisse


In [12]:
#Formulierung der Abfrage: 
myquery = sru_dnb('tit=Klimawandel and jhr=2005') #Aufruf der Funktion 'sru-dnb' mit der Abfrage 'tit=Klimawandel...'
print(len(myquery), "Ergebnisse")

29 Ergebnisse


Da im Vorfeld meist nicht bekannt ist, wieviele Ergebnisse eine Suchanfrage findet und damit zu rechnen ist, dass häufig Ergebnismengen von über 100 Treffern vorkommen, kann die Funktion nun noch etwas angepasst werden. Dazu wird zunächst abgefragt, wieviele Treffer gefunden wurden und dann entschieden: Werden bis zu 100 Treffer gemeldet, gibt es keinen Änderungsbedarf und die Funktion kann so einfach durchlaufen und die gefundenen Treffer zurückgeben. 
Werden jedoch mehr als 100 Treffer gefunden, wird die Anfrage in 100er Schritte aufgeteilt und die Ergebnisse jeweils zwischengespeichert. Dies geschieht mit Hilfe einer Schleife, die nach jeder neuen Abfrage überprüft, ob noch weitere Treffer zu holen sind und diese in 100er Schritten "einsammelt". Sobald alle Teile der Anfrage verarbeitet wurden, wird dann das gesammelte Ergebnis übergeben. Die Funktion sieht dann folgendermaßen aus: 

In [1]:
def sru_dnb(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content, features="xml")
    records = response.find_all('record')  
    
    
    if len(records) <= 100: # wurden maximal 100 Treffer gefunden? Wenn ja, erfolgt direkt die Rückgabe.
        return records
    
    elif len(records) > 100:                   # wurden mehr als 100 Treffer gefunden, wird hier die Schleife gestartet.
        num_results = 100
        i = 101
        
        while num_results == 100:
            
            parameter.update({'startRecord': i})           
            r = requests.get(dnb_url, params=parameter)
            xml = soup(r.content, features="xml")
            new_records = xml.find_all('record')
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records
    
    else:
        print("Seomthing went wrong.")

Diese Funktion wird auch bereits im Tutorial ["Wie können Daten mittels der SRU-Schnittstelle abgerufen werden?"](https://www.dnb.de/DE/Professionell/Services/WissenschaftundForschung/DNBLab/dnblab_node.html#doc731014bodyText4) verwendet. 

Eine Abfrage für das Titelstichwort "Klimawandel", kombiniert nun mit den Jahr 2019, ergibt nun folgende Treffermenge: 

In [2]:
myquery = sru_dnb('tit=Klimawandel and jhr=2019')
print(len(myquery), "Ergebnisse")

NameError: name 'requests' is not defined

Die so abgerufenen Treffer können entweder direkt weiterverarbeitet oder lokal zwischengespeichert werden. Der folgende Codeblock erstellt eine entsprechende Datei und legt den Inhalt der Abfrage dort als XML ab: 

In [15]:
with open('sru_abfrage_klimawandel.xml', 'w', encoding="utf-8") as f:
    f.write(str(myquery))

Hinweis: Wie oben bereits erwähnt, entnimmt die Schleife jeweils die bibliografischen Datensätze aus den XML-Antworten der Schnittstelle und legt diese in eine Liste ab. Es handelt sich daher bei der Variable *myquery* nicht um valides XML, da ein Root-Element fehlt. Für die weitere Verarbeitung der abgerufenen Datensätze zu Analysezwecken ist dies jedoch unerheblich. Bei Bedarf kann der Code entsprechend angepasst werden, dies ist allerdings nicht mehr Teil dieses Tutorials. 